In [2]:
import sys
print(sys.version)

3.6.0 (default, Jan  1 2017, 12:26:13) 
[GCC 6.3.1 20161221 (Red Hat 6.3.1-1)]


---

## Format strings

In [3]:
a = 1
b = 2

print('%s + %s = %s' % (a, b, a+b))

1 + 2 = 3


In [4]:
print('a is %s' % a)

a is 1


In [6]:
z = (1, 2)
print('z is %s' % z)

TypeError: not all arguments converted during string formatting

In [7]:
print('%(a)s + %(b)s = %(result)s' % {'a': a, 'b': b, 'result': a+b})

1 + 2 = 3


In [8]:
print('{} + {} = {}'.format(a, b, a+b))

1 + 2 = 3


In [9]:
print('{a} + {b} = {result}'.format(a=a, b=b, result=a+b))

1 + 2 = 3


In [10]:
print(f'{a} + {b} = {a+b}')

1 + 2 = 3


---

## Underscores in numeric literals

In [11]:
10000000

10000000

In [12]:
10_000_000

10000000

---

## Dicts are *not* ordered – but some things are

In [54]:
print({'p': 1, 'y': 2, 't': 3, 'h': 4, 'o': 5, 'n': 6})

{'p': 1, 'y': 2, 't': 3, 'h': 4, 'o': 5, 'n': 6}


In [56]:
def printargs(**kwargs):
    for key, value in kwargs.items():
        print(key, value)

printargs(p=1, y=2, t=3, h=4, o=5, n=6)

p 1
y 2
t 3
h 4
o 5
n 6


In [68]:
class C:
    p = 1
    y = 2
    t = 3
    h = 4
    o = 5
    n = 6

print(C.__dict__)

{'__module__': '__main__', 'p': 1, 'y': 2, 't': 3, 'h': 4, 'o': 5, 'n': 6, '__dict__': <attribute '__dict__' of 'C' objects>, '__weakref__': <attribute '__weakref__' of 'C' objects>, '__doc__': None}


---

## ModuleNotFoundError

In [136]:
import secret

ModuleNotFoundError: No module named 'secret'

---

## Psst! `secrets`

In [47]:
import secrets

In [52]:
secrets.token_urlsafe()

'0ee_7OPs5LgZ3yZ5gygzXIplLGNtVB5Qe-62qBsCX4c'

---

## Enum: Flags and auto-values

In [140]:
import enum

In [141]:
class Flip(enum.Flag):
    horizontal = enum.auto()
    vertical = enum.auto()
    diagonal = enum.auto()

In [149]:
Flip.horizontal

<Flip.horizontal: 1>

In [145]:
Flip.horizontal | Flip.diagonal

<Flip.diagonal|horizontal: 5>

In [148]:
Flip(5)

<Flip.diagonal|horizontal: 5>

In [151]:
Flip(~2)

<Flip.diagonal|horizontal: 5>

In [152]:
~Flip.vertical

<Flip.diagonal|horizontal: 5>

---

## Path-like Objects

In [159]:
class RandomPath:
    def __fspath__(self):
        return '/dev/random'

with open(RandomPath(), 'rb') as f:
    print(f.read(10))

b'(\xbf\x14\x0c(\xdeNiQ\xbd'


In [160]:
import pathlib

path = pathlib.Path('/') / 'dev' / 'random'

path

PosixPath('/dev/random')

In [161]:
with open(path, 'rb') as f:
    print(f.read(10))

b'\x1b\xa3\xc9\xe5q\x10\x81\xf3%\xd2'


In [162]:
with path.open('rb') as f:
    print(f.read(10))

b'\xdc\x1eY$\xcc\x15\xc6e\xad,'


In [163]:
import os
os.fspath(RandomPath())

'/dev/random'

---

## Async generators & comprehensions

In [41]:
import asyncio
async def countdown():
    for i in reversed(range(5)):
        yield i
        await asyncio.sleep(1)
    yield('Liftoff!')

In [42]:
async def consumer():
    async for i in countdown():
        print(i)

In [43]:
loop = asyncio.get_event_loop()
loop.run_until_complete(consumer())

4
3
2
1
0
Liftoff!


In [46]:
async def example():
    return [i async for i in countdown()]

loop = asyncio.get_event_loop()
loop.run_until_complete(example())

[4, 3, 2, 1, 0, 'Liftoff!']

Plus lots more `asyncio` changes. And, `asyncio` is not provisional anymore – finally stable!
(And it's also up to 30% faster)

---

## Nicer access to `re` groups

In [175]:
import re
match = re.match(r'(?P<name>.*) (?P<major>\d+)\.(?P<minor>\d+)', 'Python 3.6')

In [176]:
match['name']

'Python'

In [177]:
match[0]

'Python 3.6'

---

## Variable annotations

In [26]:
def add(a: int, b: int) -> int:
    return a + b

In [27]:
print(add(1, 2))

3


In [28]:
print(add('a', 'b'))

ab


In [19]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [23]:
%%cython
cpdef int cadd(int a, int b):
    return a + b

In [24]:
print(cadd(1, 2))

3


In [25]:
print(cadd("a", "b"))

TypeError: an integer is required

In [29]:
number = 1

In [30]:
number: int = 1

In [31]:
number: int

In [92]:
class C():
    number: int

In [93]:
C.__annotations__

{'number': int}

The (provisional) `typing` module also includes some additions.

---

## Simpler Customization of Class Creation

In [58]:
class Registered:
    registry = []
    
    def __init_subclass__(cls, **kwargs):
        cls.registry.append(cls)

class Plugin1(Registered):
    pass

class Plugin2(Registered):
    pass

In [59]:
Registered.registry

[__main__.Plugin1, __main__.Plugin2]

---

## Naming Descriptors

In [78]:
class Column:
    def __get__(self, instance, cls):
        return instance.data[self.name]

    def __set_name__(self, instance, name):
        self.name = name

class Row:
    def __init__(self, data):
        self.data = data
    
    product = Column()
    price = Column()


In [79]:
row = Row({'product': 'apple', 'price': 30})

print(row.product)
print(row.price)


apple
30


---

## RecursionError display improvement

---

### The Adding of τ

In [137]:
import math
math.tau

6.283185307179586

In [138]:
math.tau == math.pi * 2

True

---

## Unicode 9.0

In [114]:
print('\N{BAT} \N{LIZARD} \N{BUTTERFLY} \N{OWL}')

🦇 🦎 🦋 🦉


In [118]:
import unicodedata
unicodedata.name('🦋')

'BUTTERFLY'

---

## Plus

* 16-bit wordcode
* Security improvements
* Protocol "anti-registration" for special methods
* PYTHONMALLOC; Frame evaluation API; SystemTap support; ResourceWarning source
* Progress on Android support
* `asyncio` now stable; improvements & speedups;  `asynchat` & `asyncore` deprecated
* read: "What's New in Python 3.6"

---

## Announcements

* DevConf tomorrow
* `pyvo.cz` feedback form -- pyvo.cz/feedback
* We need you!
* Sprint